Functions to format the queries properly for the endpoint.

In [1]:
import re

def format_query(query):
    PREFIXES = """PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX yago: <http://yago-knowledge.org/resource/>
PREFIX y2geor: <http://kr.di.uoa.gr/yago2geo/resource/>
PREFIX y2geoo: <http://kr.di.uoa.gr/yago2geo/ontology/>
PREFIX strdf: <http://strdf.di.uoa.gr/ontology#>
PREFIX uom: <http://www.opengis.net/def/uom/OGC/1.0/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>"""
    
    query = PREFIXES + ' ' + query
    
    query = query.replace('strdf:within', 'geof:sfWithin')
    query = query.replace('strdf:contains', 'geof:sfContains')
    query = query.replace('strdf:overlaps', 'geof:sfOverlaps')
    query = query.replace('strdf:distance', 'geof:sfDistance')
    
    # Use regex to find and replace strdf:buffer patterns
    query = re.sub(r'strdf:buffer\((\?\w+),\s*\d+,\s*uom:\w+\)', r'\1', query)
    
    return query

In [2]:
def gost_materialize_query(query: str):
    data = {
        "query": query
    }

    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.post("materialize-api", headers=headers, data=json.dumps(data))
    
    if response.status_code == 200:
        return response.text
    else:
        print("Materialize failed:", response.text)
        return (query)

Function that sends query to the endpoint and returns the result.

In [3]:
import requests
import pandas as pd
from io import StringIO

def graphdb_send_request(query, endpoint_url="url", accept_format='application/sparql-results+json'):
    """
    Sends a SPARQL query to a GraphDB endpoint.

    :param query: SPARQL query to be sent
    :param endpoint_url: URL of the GraphDB SPARQL endpoint
    :param accept_format: Desired response format (default is JSON)
    :return: Response from the endpoint
    """
    # Format the query, this means add the correct prefixes and fix some endpoint issues with regex.
    query = format_query(query)
    original_query = query
    query = gost_materialize_query(query)
    
    headers = {
        'Accept': accept_format,
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        'query': query
    }
    
    try:
        response = requests.post(endpoint_url, headers=headers, data=data, auth=requests.auth.HTTPBasicAuth('aaaa', 'ooooooeooo'))

        if response.status_code == 200:
            if accept_format == 'application/sparql-results+json':
#                 print(response.json())
                json_response = response.json()
                return convert_json_to_csv(json_response)
            else:
#                 print(response.text)
                return response.text
        else:
            response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print("HTTP error (most likely invalid query)")
        #print(query)
        #print(err)
    except Exception as err:
        print(err)
        print("Endpoint error ENDPOINT DOWN")
        
def convert_json_to_csv(json_data):
    """
    Converts JSON data to CSV format.

    :param json_data: JSON data to be converted
    :return: CSV formatted data as a string
    """
    if 'boolean' in json_data:
        # Handling boolean result
        headers = ['value']
        rows = [[json_data['boolean']]]
    else:
        # Extracting header and rows from JSON response
        headers = json_data['head']['vars']
        rows = [{var: result.get(var, {}).get('value', '') for var in headers} for result in json_data['results']['bindings']]
    
    # Creating DataFrame and converting to CSV
    df = pd.DataFrame(rows, columns=headers)
    csv_output = StringIO()
    df.to_csv(csv_output, index=False)
    
    return csv_output.getvalue()

Inference function.

In [4]:
import torch

def run_inference(model, tokenizer, prompt):
    results = []
    
    if tokenizer == None:
        # Generate output
        with torch.no_grad():
            outputs = model(prompt)
            
        # Decode and print output
        #print("Prompt:", prompt)
        #print("Generated text:" + outputs + "\n")
        results.append("Generated text:" + outputs)
    else:
        # Move model to GPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.eval()  # Set model to evaluation mode
            
        # Tokenize prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
            
        # Generate output
        with torch.no_grad():
            outputs = model.generate(**inputs, 
                            max_new_tokens=800,  # Set a maximum length for generated text
                            #do_sample=True,  # Enable sampling
                            #top_k=7,        # Top-k sampling
                            #top_p=0.1,      # Top-p sampling (nucleus sampling)
                            #num_return_sequences=1,
                            #repetition_penalty=1, # No penalty for instruction tuned models.
                            repetition_penalty=1.2, # Penalty on repeating tokens.
                            eos_token_id=tokenizer.eos_token_id,  # Specify EOS token ID
                            pad_token_id=tokenizer.pad_token_id  # Specify PAD token ID
                            )
        
        # Extract generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Remove the prompt text
        prompt_length = len(prompt)
        generated_text = generated_text[prompt_length:]

        # Decode and print output
        #print("Prompt:", prompt)
        #print(generated_text)
        results.append(generated_text)
    
    # Clear model from RAM
    del model
    torch.cuda.empty_cache()
    
    return results
    
def compare_strings(str1, str2):
    # Split strings into words
    words1 = str1.split()
    words2 = str2.split()
    
    # Determine the length of the shorter list
    min_length = min(len(words1), len(words2))
    
    # Compare words from both lists
    for i in range(min_length):
        if words1[i] != words2[i]:
            print(f"Difference found: '{words1[i]}' != '{words2[i]}'")
    
    # Handle any extra words in the longer list
    if len(words1) > min_length:
        for i in range(min_length, len(words1)):
            print(f"Extra word in first string: '{words1[i]}'")
    
    if len(words2) > min_length:
        for i in range(min_length, len(words2)):
            print(f"Extra word in second string: '{words2[i]}'")
            
def find_prompt_end_position(generated_text, prompt):
    # Remove all spaces from the sentence and paragraph
    clean_paragraph = generated_text.replace(' ', '')
    clean_sentence = prompt.replace(' ', '')
    
    # Find the start position of the clean sentence in the clean paragraph
    start_position = clean_paragraph.find(clean_sentence)
    
    if start_position == -1:
        print("MAN")
        return -1  # Sentence not found in the paragraph
    
    # Calculate the end position in the original paragraph
    sentence_length = len(clean_sentence)
    end_position_clean = start_position + sentence_length
    
    # Convert the end position to the original paragraph with spaces
    current_position = 0
    end_position_original = 0
    for char in generated_text:
        if char != ' ':
            current_position += 1
        end_position_original += 1
        if current_position == end_position_clean:
            break
    
    return end_position_original

def Quantized_Inference(model, tokenizer, prompt):
    results = []
    
    # Move model to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()  # Set model to evaluation mode
            
    # Tokenize prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
            
    # Generate output
    with torch.no_grad():
        outputs = model.generate(**inputs, 
                            max_new_tokens=250,  # Set a maximum length for generated text
                            #do_sample=True,  # Enable sampling
                            #top_k=7,        # Top-k sampling
                            #top_p=0.1,      # Top-p sampling (nucleus sampling)
                            #num_return_sequences=1,
                            repetition_penalty=1.2, # Penalty on repeating tokens.
                            eos_token_id=tokenizer.eos_token_id,  # Specify EOS token ID
                            pad_token_id=tokenizer.pad_token_id  # Specify PAD token ID
                            )
        
    # Extract generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remove the prompt text
    prompt_length = find_prompt_end_position(generated_text, prompt)
    generated_text = generated_text[prompt_length:]
    
    # Clear model from RAM
    del model
    torch.cuda.empty_cache()
    
    return generated_text

In [5]:
def insert_space_before_punctuation(query):
    # Function to replace punctuation outside of URIs
    def replace_punctuation(match):
        text = match.group(0)
        if text.startswith('<') and text.endswith('>'):
            # This is a URI, don't modify it
            return text
        else:
            # Add spaces before punctuation
            return text.replace('?', ' ?').replace('.', ' .')

    # Regex pattern to match URIs and text between them
    pattern = r'(<[^>]+>|[^<>]+)'

    # Apply the replacement function
    processed_query = re.sub(pattern, replace_punctuation, query)

    return processed_query

In [6]:
import re

def extract_uris(query):
    # Regular expression to match both fully expanded and prefixed URIs
    uri_pattern = r'<([^>]+)>|(\b[a-zA-Z0-9_]+):([a-zA-Z0-9_]+)'
    
    uris = []
    matches = re.findall(uri_pattern, query)
    for match in matches:
        if match[0]:  # Fully expanded URI
            uris.append(match[0])
        else:  # Prefixed URI
            uris.append(f"{match[1]}:{match[2]}")
    return uris

def expand_uris(query, prefix_dict):
    prefixed_pattern = r'(\b[a-zA-Z0-9_]+):([a-zA-Z0-9_]+)'
    expanded_pattern = r'<([^>]+)>'
    
    expanded_uris = []
    
    # Find and expand prefixed URIs
    matches = re.findall(prefixed_pattern, query)
    for prefix, suffix in matches:
        if prefix in prefix_dict:
            expanded_uris.append(f"{prefix_dict[prefix]}{suffix}")
        else:
            expanded_uris.append(f"{prefix}:{suffix}")
    
    # Find and add already expanded URIs
    matches = re.findall(expanded_pattern, query)
    for uri in matches:
        expanded_uris.append(uri)
    
    return expanded_uris

prefix_dict = {
    'geo': 'http://www.opengis.net/ont/geosparql#',
    'osm': 'http://www.openstreetmap.org/ontology#',
    'xsd': 'http://www.w3.org/2001/XMLSchema#',
    'geof': 'http://www.opengis.net/def/function/geosparql/',
    'uom': 'http://www.opengis.net/def/uom/OGC/1.0/'
}

Function to generate queries with 6 example few shot learning and uri injection. (best experimental results)

In [7]:
def get_plain_zero_shot_prompt(user_prompt):
    prompt = f"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph. The query should be enclosed by three backticks on new lines, denoting that it is a code block.
Human: {user_prompt}
Generator: ```"""
    return prompt

def get_plain_three_shot_prompt(user_prompt):
    prompt = f"""Human: In Breckland district, which forests are south of streams?
Generator: ```SELECT DISTINCT ?forest WHERE {{ yago:Breckland_District geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?forest rdf:type y2geoo:OSM_forest . ?forest geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . ?stream rdf:type y2geoo:OSM_stream . ?stream geo:hasGeometry ?o3 . ?o3 geo:asWKT ?geoWKT3 . FILTER (strdf:within(?geoWKT2, ?geoWKT1) && strdf:within(?geoWKT3, ?geoWKT1) && strdf:below(?geoWKT2, ?geoWKT3)) }}```
Human: How many streams intersect with lakes?
Generator: ```SELECT (COUNT (DISTINCT ?p1) as ?streams) WHERE {{ ?p1 rdf:type y2geoo:OSM_stream; geo:hasGeometry ?p1geo. ?p1geo geo:asWKT ?p1WKT. ?p2 rdf:type y2geoo:OSM_lake; geo:hasGeometry ?p2geo. ?p2geo geo:asWKT ?p2WKT. FILTER(geof:sfIntersects(?p1WKT, ?p2WKT)) }}```
Human: Which Municipalities are on Thessaly's border?
Generator: ```SELECT distinct ?rg where {{ yago:Thessaly geo:hasGeometry ?tgeo . ?tgeo geo:asWKT ?tgWKT . ?rg rdf:type y2geoo:GAG_Municipality . ?rg geo:hasGeometry ?rggeo . ?rggeo geo:asWKT ?rgWKT . FILTER (strdf:touches(?tgWKT,?rgWKT)) . }}```
Human: {user_prompt}
Generator: ```"""
    return prompt

def get_zero_shot_uri_prompt(user_prompt, uris):
    prompt = f"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph. The query should be enclosed by three backticks on new lines, denoting that it is a code block.
Human: {user_prompt}
The generator must use these URIs to answer the question: {uris}
Generator: ```"""
    return prompt

def get_zero_shot_uri_geos_prompt(user_prompt, uris):
    prompt = f"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph. The query should be enclosed by three backticks on new lines, denoting that it is a code block.
The resulting query may have to be in GeoSPARQL. The GeoSPARQL ontology is defined by:
URI: http://www.opengis.net/ont/geosparql
Classes: Feature, Feature Collection, Geometry, Geometry Collection, Spatial Object, Spatial Object Collection
Object Properties: default geometry, contains, covered by, covers, disjoint, equals, inside, meet, overlap, has area, has bounding box, has centroid, has default geometry, has geometry, has length, has perimeter length, has size, has spatial accuracy, has spatial resolution, has volume, disconnected, externally connected, equals, non-tangential proper part, non-tangential proper part inverse, partially overlapping, tangential proper part, tangential proper part inverse, contains, crosses, disjoint, equals, intersects, overlaps, touches, within
Datatype Properties: as DGGS, as GML, as GeoJSON, as KML, as WKT, coordinate dimension, dimension, has area in square meters, has length in meters, has perimeter length in meters, has metric size, has spatial accuracy in meters, has spatial resolution in meters, has volume in cubic meters, has serialization, is empty, is simple, spatial dimension
Human: {user_prompt}
The generator must use these URIs to answer the question: {uris}
Generator: ```"""
    return prompt

def get_three_shot_uri_prompt(user_prompt, uris):
    prompt = f"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph. The query should be enclosed by three backticks on new lines, denoting that it is a code block.
Human: In Breckland district, which forests are south of streams?
The generator must use these URIs to answer the question: ['yago:Breckland_District', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:OSM_forest', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:OSM_stream', 'geo:hasGeometry', 'geo:asWKT', 'strdf:within', 'strdf:within', 'strdf:below']
Generator: ```SELECT DISTINCT ?forest WHERE {{ yago:Breckland_District geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?forest rdf:type y2geoo:OSM_forest . ?forest geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . ?stream rdf:type y2geoo:OSM_stream . ?stream geo:hasGeometry ?o3 . ?o3 geo:asWKT ?geoWKT3 . FILTER (strdf:within(?geoWKT2, ?geoWKT1) && strdf:within(?geoWKT3, ?geoWKT1) && strdf:below(?geoWKT2, ?geoWKT3)) }}```
Human: How many streams intersect with lakes?
The generator must use these URIs to answer the question: ['rdf:type', 'y2geoo:OSM_stream', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:OSM_lake', 'geo:hasGeometry', 'geo:asWKT', 'geof:sfIntersects']
Generator: ```SELECT (COUNT (DISTINCT ?p1) as ?streams) WHERE {{ ?p1 rdf:type y2geoo:OSM_stream; geo:hasGeometry ?p1geo. ?p1geo geo:asWKT ?p1WKT. ?p2 rdf:type y2geoo:OSM_lake; geo:hasGeometry ?p2geo. ?p2geo geo:asWKT ?p2WKT. FILTER(geof:sfIntersects(?p1WKT, ?p2WKT)) }}```
Human: Which Municipalities are on Thessaly's border?
The generator must use these URIs to answer the question: ['yago:Thessaly', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:GAG_Municipality', 'geo:hasGeometry', 'geo:asWKT', 'strdf:touches']
Generator: ```SELECT distinct ?rg where {{ yago:Thessaly geo:hasGeometry ?tgeo . ?tgeo geo:asWKT ?tgWKT . ?rg rdf:type y2geoo:GAG_Municipality . ?rg geo:hasGeometry ?rggeo . ?rggeo geo:asWKT ?rgWKT . FILTER (strdf:touches(?tgWKT,?rgWKT)) . }}```
Human: {user_prompt}
The generator must use these URIs to answer the question: {uris}
Generator: ```"""
    return prompt

def get_six_shot_uri_prompt(user_prompt, uris):
    prompt = f"""Human: In Breckland district, which forests are south of streams?
The generator must use these URIs to answer the question: ['yago:Breckland_District', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:OSM_forest', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:OSM_stream', 'geo:hasGeometry', 'geo:asWKT', 'strdf:within', 'strdf:within', 'strdf:below']
Generator: ```SELECT DISTINCT ?forest WHERE {{ yago:Breckland_District geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?forest rdf:type y2geoo:OSM_forest . ?forest geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . ?stream rdf:type y2geoo:OSM_stream . ?stream geo:hasGeometry ?o3 . ?o3 geo:asWKT ?geoWKT3 . FILTER (strdf:within(?geoWKT2, ?geoWKT1) && strdf:within(?geoWKT3, ?geoWKT1) && strdf:below(?geoWKT2, ?geoWKT3)) }}```
Human: How many streams intersect with lakes?
The generator must use these URIs to answer the question: ['rdf:type', 'y2geoo:OSM_stream', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:OSM_lake', 'geo:hasGeometry', 'geo:asWKT', 'geof:sfIntersects']
Generator: ```SELECT (COUNT (DISTINCT ?p1) as ?streams) WHERE {{ ?p1 rdf:type y2geoo:OSM_stream; geo:hasGeometry ?p1geo. ?p1geo geo:asWKT ?p1WKT. ?p2 rdf:type y2geoo:OSM_lake; geo:hasGeometry ?p2geo. ?p2geo geo:asWKT ?p2WKT. FILTER(geof:sfIntersects(?p1WKT, ?p2WKT)) }}```
Human: Which Municipalities are on Thessaly's border?
The generator must use these URIs to answer the question: ['yago:Thessaly', 'geo:hasGeometry', 'geo:asWKT', 'rdf:type', 'y2geoo:GAG_Municipality', 'geo:hasGeometry', 'geo:asWKT', 'strdf:touches']
Generator: ```SELECT distinct ?rg where {{ yago:Thessaly geo:hasGeometry ?tgeo . ?tgeo geo:asWKT ?tgWKT . ?rg rdf:type y2geoo:GAG_Municipality . ?rg geo:hasGeometry ?rggeo . ?rggeo geo:asWKT ?rgWKT . FILTER (strdf:touches(?tgWKT,?rgWKT)) . }}```
Human: Which is the largest island in Ireland?
The generator must use these URIs to answer the question: ['strdf:area', 'yago:Republic_of_Ireland', 'geo:hasGeometry', 'geo:asWKT', 'y2geoo:OSM_island', 'geo:hasGeometry', 'geo:asWKT', 'geof:sfContains']
Generator: ```select distinct ?x (strdf:area(?lWKT) as ?area) where {{ yago:Republic_of_Ireland geo:hasGeometry ?geom . ?geom geo:asWKT ?mWKT . ?lake a y2geoo:OSM_island . ?lake geo:hasGeometry ?geol . ?geol geo:asWKT ?lWKT . FILTER (geof:sfContains(?mWKT, ?lWKT)) }} ORDER BY (?area) LIMIT 1```
Human: Is Crete south of Thessaly?
The generator must use these URIs to answer the question: ['http://yago-knowledge.org/resource/Crete', 'geo:hasGeometry', 'http://yago-knowledge.org/resource/Thessaly', 'geo:hasGeometry', 'geo:asWKT', 'geo:asWKT', 'strdf:below']
Generator: ```ASK {{ <http://yago-knowledge.org/resource/Crete> geo:hasGeometry ?geo1 . <http://yago-knowledge.org/resource/Thessaly> geo:hasGeometry ?geo2 . ?geo1 geo:asWKT ?geoWKT1 . ?geo2 geo:asWKT ?geoWKT2 . FILTER(strdf:below(?geoWKT1, ?geoWKT2)) }}```
Human: What is the population of Northern Ireland?
The generator must use these URIs to answer the question: ['xsd:integer', 'yago:Northern_Ireland', 'yago:hasPopulation']
Generator: ```SELECT (xsd:integer (?population) as ?pop) WHERE {{ yago:Northern_Ireland yago:'hasPopulation ?population. }}```
Human: {user_prompt}
The generator must use these URIs to answer the question: {uris}
Generator: ```"""
    
    return prompt

def get_six_shot_auri_prompt(user_prompt, uris):
    prompt = f"""Human: In Breckland district, which forests are south of streams?
The generator may use these URIs to answer the question: ['yago:Breckland_District', 'y2geoo:OS_DistrictWard', 'y2geoo:OSM_forest', 'y2geoo:OSM_stream']
Generator: ```SELECT DISTINCT ?forest WHERE {{ yago:Breckland_District geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?forest rdf:type y2geoo:OSM_forest . ?forest geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . ?stream rdf:type y2geoo:OSM_stream . ?stream geo:hasGeometry ?o3 . ?o3 geo:asWKT ?geoWKT3 . FILTER (strdf:within(?geoWKT2, ?geoWKT1) && strdf:within(?geoWKT3, ?geoWKT1) && strdf:below(?geoWKT2, ?geoWKT3)) }}```
Human: How many streams intersect with lakes?
The generator may use these URIs to answer the question: ['y2geoo:OSM_lake']
Generator: ```SELECT (COUNT (DISTINCT ?p1) as ?streams) WHERE {{ ?p1 rdf:type y2geoo:OSM_stream; geo:hasGeometry ?p1geo. ?p1geo geo:asWKT ?p1WKT. ?p2 rdf:type y2geoo:OSM_lake; geo:hasGeometry ?p2geo. ?p2geo geo:asWKT ?p2WKT. FILTER(geof:sfIntersects(?p1WKT, ?p2WKT)) }}```
Human: Which Municipalities are on Thessaly's border?
The generator may use these URIs to answer the question: []
Generator: ```SELECT distinct ?rg where {{ yago:Thessaly geo:hasGeometry ?tgeo . ?tgeo geo:asWKT ?tgWKT . ?rg rdf:type y2geoo:GAG_Municipality . ?rg geo:hasGeometry ?rggeo . ?rggeo geo:asWKT ?rgWKT . FILTER (strdf:touches(?tgWKT,?rgWKT)) . }}```
Human: Which is the largest island in Ireland?
The generator may use these URIs to answer the question: ['y2geoo:OSM_island', 'y2geoo:OSM_stream']
Generator: ```select distinct ?x (strdf:area(?lWKT) as ?area) where {{ yago:Republic_of_Ireland geo:hasGeometry ?geom . ?geom geo:asWKT ?mWKT . ?lake a y2geoo:OSM_island . ?lake geo:hasGeometry ?geol . ?geol geo:asWKT ?lWKT . FILTER (geof:sfContains(?mWKT, ?lWKT)) }} ORDER BY (?area) LIMIT 1```
Human: Is Crete south of Thessaly?
The generator may use these URIs to answer the question: []
Generator: ```ASK {{ yago:Crete geo:hasGeometry ?geo1 . yago:Thessaly geo:hasGeometry ?geo2 . ?geo1 geo:asWKT ?geoWKT1 . ?geo2 geo:asWKT ?geoWKT2 . FILTER(strdf:below(?geoWKT1, ?geoWKT2)) }}```
Human: What is the population of Northern Ireland?
The generator may use these URIs to answer the question: ['yago:Ireland']
Generator: ```SELECT (xsd:integer (?population) as ?pop) WHERE {{ yago:Northern_Ireland yago:'hasPopulation ?population. }}```
Human: {user_prompt}
The generator may use these URIs to answer the question: {uris}
Generator: ```"""
    
    return prompt

def get_plain_six_shot_prompt(prompt):
    prompt = f"""Human: In Breckland district, which forests are south of streams?
Generator: ```SELECT DISTINCT ?forest WHERE {{ yago:Breckland_District geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?forest rdf:type y2geoo:OSM_forest . ?forest geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . ?stream rdf:type y2geoo:OSM_stream . ?stream geo:hasGeometry ?o3 . ?o3 geo:asWKT ?geoWKT3 . FILTER (strdf:within(?geoWKT2, ?geoWKT1) && strdf:within(?geoWKT3, ?geoWKT1) && strdf:below(?geoWKT2, ?geoWKT3)) }}```
Human: How many streams intersect with lakes?
Generator: ```SELECT (COUNT (DISTINCT ?p1) as ?streams) WHERE {{ ?p1 rdf:type y2geoo:OSM_stream; geo:hasGeometry ?p1geo. ?p1geo geo:asWKT ?p1WKT. ?p2 rdf:type y2geoo:OSM_lake; geo:hasGeometry ?p2geo. ?p2geo geo:asWKT ?p2WKT. FILTER(geof:sfIntersects(?p1WKT, ?p2WKT)) }}```
Human: Which Municipalities are on Thessaly's border?
Generator: ```SELECT distinct ?rg where {{ yago:Thessaly geo:hasGeometry ?tgeo . ?tgeo geo:asWKT ?tgWKT . ?rg rdf:type y2geoo:GAG_Municipality . ?rg geo:hasGeometry ?rggeo . ?rggeo geo:asWKT ?rgWKT . FILTER (strdf:touches(?tgWKT,?rgWKT)) . }}```
Human: Which is the largest island in Ireland?
Generator: ```select distinct ?x (strdf:area(?lWKT) as ?area) where {{ yago:Republic_of_Ireland geo:hasGeometry ?geom . ?geom geo:asWKT ?mWKT . ?lake a y2geoo:OSM_island . ?lake geo:hasGeometry ?geol . ?geol geo:asWKT ?lWKT . FILTER (geof:sfContains(?mWKT, ?lWKT)) }} ORDER BY (?area) LIMIT 1```
Human: Is Crete south of Thessaly?
Generator: ```ASK {{ <http://yago-knowledge.org/resource/Crete> geo:hasGeometry ?geo1 . <http://yago-knowledge.org/resource/Thessaly> geo:hasGeometry ?geo2 . ?geo1 geo:asWKT ?geoWKT1 . ?geo2 geo:asWKT ?geoWKT2 . FILTER(strdf:below(?geoWKT1, ?geoWKT2)) }}```
Human: What is the population of Northern Ireland?
Generator: ```SELECT (xsd:integer (?population) as ?pop) WHERE {{ yago:Northern_Ireland yago:'hasPopulation ?population. }}```
Human: {prompt}
Generator: ```"""
    
    return prompt    
    
def get_uri_prompt(user_prompt, uris):
    prompt = f"""Human: {user_prompt}
The generator must use these URIs to answer the question: {uris}
Generator: ```"""
    return prompt

def get_plain_three_shot_prompt(user_prompt):
    prompt = f"""Human: In Breckland district, which forests are south of streams?
Generator: ```SELECT DISTINCT ?forest WHERE {{ yago:Breckland_District geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?forest rdf:type y2geoo:OSM_forest . ?forest geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . ?stream rdf:type y2geoo:OSM_stream . ?stream geo:hasGeometry ?o3 . ?o3 geo:asWKT ?geoWKT3 . FILTER (strdf:within(?geoWKT2, ?geoWKT1) && strdf:within(?geoWKT3, ?geoWKT1) && strdf:below(?geoWKT2, ?geoWKT3)) }}```
Human: How many streams intersect with lakes?
Generator: ```SELECT (COUNT (DISTINCT ?p1) as ?streams) WHERE {{ ?p1 rdf:type y2geoo:OSM_stream; geo:hasGeometry ?p1geo. ?p1geo geo:asWKT ?p1WKT. ?p2 rdf:type y2geoo:OSM_lake; geo:hasGeometry ?p2geo. ?p2geo geo:asWKT ?p2WKT. FILTER(geof:sfIntersects(?p1WKT, ?p2WKT)) }}```
Human: Which Municipalities are on Thessaly's border?
Generator: ```SELECT distinct ?rg where {{ yago:Thessaly geo:hasGeometry ?tgeo . ?tgeo geo:asWKT ?tgWKT . ?rg rdf:type y2geoo:GAG_Municipality . ?rg geo:hasGeometry ?rggeo . ?rggeo geo:asWKT ?rgWKT . FILTER (strdf:touches(?tgWKT,?rgWKT)) . }}```
Human: {user_prompt}
Generator: ```"""
    return prompt

Inference with GPT-4 Turbo.

In [ ]:
pip install openai==0.28

In [9]:
greatjobman = f"""Human: In Breckland district, which forests are south of streams?
Generator: ```SELECT DISTINCT ?forest WHERE {{ yago:Breckland_District geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?forest rdf:type y2geoo:OSM_forest . ?forest geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . ?stream rdf:type y2geoo:OSM_stream . ?stream geo:hasGeometry ?o3 . ?o3 geo:asWKT ?geoWKT3 . FILTER (strdf:within(?geoWKT2, ?geoWKT1) && strdf:within(?geoWKT3, ?geoWKT1) && strdf:below(?geoWKT2, ?geoWKT3)) }}```
Human: How many streams intersect with lakes?
Generator: ```SELECT (COUNT (DISTINCT ?p1) as ?streams) WHERE {{ ?p1 rdf:type y2geoo:OSM_stream; geo:hasGeometry ?p1geo. ?p1geo geo:asWKT ?p1WKT. ?p2 rdf:type y2geoo:OSM_lake; geo:hasGeometry ?p2geo. ?p2geo geo:asWKT ?p2WKT. FILTER(geof:sfIntersects(?p1WKT, ?p2WKT)) }}```
Human: Which Municipalities are on Thessaly's border?
Generator: ```SELECT distinct ?rg where {{ yago:Thessaly geo:hasGeometry ?tgeo . ?tgeo geo:asWKT ?tgWKT . ?rg rdf:type y2geoo:GAG_Municipality . ?rg geo:hasGeometry ?rggeo . ?rggeo geo:asWKT ?rgWKT . FILTER (strdf:touches(?tgWKT,?rgWKT)) . }}```
Human: Which is the largest island in Ireland?
Generator: ```select distinct ?x (strdf:area(?lWKT) as ?area) where {{ yago:Republic_of_Ireland geo:hasGeometry ?geom . ?geom geo:asWKT ?mWKT . ?lake a y2geoo:OSM_island . ?lake geo:hasGeometry ?geol . ?geol geo:asWKT ?lWKT . FILTER (geof:sfContains(?mWKT, ?lWKT)) }} ORDER BY (?area) LIMIT 1```
Human: Is Crete south of Thessaly?
Generator: ```ASK {{ <http://yago-knowledge.org/resource/Crete> geo:hasGeometry ?geo1 . <http://yago-knowledge.org/resource/Thessaly> geo:hasGeometry ?geo2 . ?geo1 geo:asWKT ?geoWKT1 . ?geo2 geo:asWKT ?geoWKT2 . FILTER(strdf:below(?geoWKT1, ?geoWKT2)) }}```
Human: What is the population of Northern Ireland?
Generator: ```SELECT (xsd:integer (?population) as ?pop) WHERE {{ yago:Northern_Ireland yago:'hasPopulation ?population. }}```
Human: What is the area of West Sussex?
Generator: ```"""

In [10]:
import openai

def generate_response(prompt):
    # Set your OpenAI API key here
    openai.api_key = 'nonononono'
    
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """You are an expert SPARQL query generator. For each question that the user supplies, you will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the Yago2Geo knowledge graph. The query should be enclosed by three backticks on new lines, denoting that it is a code block. You will not provide further details. You will also be provided with a few relevant URIs for each query, they are not necesarily correct. You must judge whether to use them or not."""},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
#     print(response.choices[0].message['content'])
    return response.choices[0].message['content']

In [11]:
import random

# Run inference and cleanup.
def generate_URI_injected_query(user_prompt, uris):
    # Direct extraction, NOTE: try expanded extraction.
#     uris = extract_uris(gt_query)
#     Shuffle the uris so their order is not as they appear in the query.
#     random.shuffle(uris)
    prompt = get_six_shot_auri_prompt(user_prompt, uris)
    
    results = generate_response(prompt)
    results = results[3:]
    
    end_index = results.find("```")
    query = results
    # Extract the substring from the start of the string up to the first occurrence of ```
    if end_index != -1:
        query = results[:end_index]
    else:
        # If ``` is not found, keep the original string
        query = results

    # Now remove the SPARQL prefix that the model adds.
    start_index = query.find("sparql")
    if start_index == 0:
        # Remove the prefix and all characters leading up to it
        query = query[start_index + len("sparql"):]

    print(f"URI QUERY: {query}")
    return query

In [12]:
import random

# Run inference and cleanup.
def generate_query(user_prompt, gt_query):
    prompt = get_plain_six_shot_prompt(user_prompt)
    
    results = generate_response(prompt)
    results = results[3:]
    
    end_index = results.find("```")
    query = results
    # Extract the substring from the start of the string up to the first occurrence of ```
    if end_index != -1:
        query = results[:end_index]
    else:
        # If ``` is not found, keep the original string
        query = results

    # Now remove the SPARQL prefix that the model adds.
    start_index = query.find("sparql")
    if start_index == 0:
        # Remove the prefix and all characters leading up to it
        query = query[start_index + len("sparql"):]

    print(f"PLAIN QUERY: {query}")
    return query

Normalize queries before passing them to the endpoint to ensure uniformity across results.

In [13]:
def normalize_variables(query):
    if query is None:
        return ""
    variable_pattern = re.compile(r"\?\w+")
    variables = variable_pattern.findall(query)
    normalized_query = query
    for i, var in enumerate(variables):
        normalized_query = normalized_query.replace(var, f"?var{i}")
    return normalized_query

To further ensure that correct results are not rejected, this custom comparisson function: 

In [14]:
def csv_to_columns(csv_data):
    rows = csv_data.strip().split('\n')
    data_rows = [row.split(',') for row in rows[1:]]  # Skip the header row
    columns = list(zip(*data_rows))  # Transpose rows to columns
    return columns

def compare_csv_columns(csv1, csv2):
    columns1 = csv_to_columns(csv1)
    columns2 = csv_to_columns(csv2)
    
    set_columns1 = {tuple(col) for col in columns1}
    set_columns2 = {tuple(col) for col in columns2}
    
    #return not set_columns1.isdisjoint(set_columns2)
    
    common_columns = set_columns1.intersection(set_columns2)
    
    if common_columns:
        return True
    else:
        return False

In [ ]:
import json, requests

# with open('/kaggle/input/aurii-concepts-dataset/concepts_dataset.json', 'r') as file:
#     original_dataset = json.load(file)
    
# with open('/kaggle/input/auto-uri-i-instances/LLM-WAT-ED-dataest.json', 'r') as file:
#     original_dataset = json.load(file)
    
with open('/kaggle/input/aurii-concepts-instances-dataset/concepts_instances_dataset.json', 'r') as file:
    original_dataset = json.load(file)
    
detailed_comp1 = 0

detailed_comp2 = 0

gt_results = []
gen_results = []

i = 0
for key in original_dataset:
    print(i)
    i+=1
#     if i < 13:
#         continue
    query = original_dataset[key]['Query']
    # Normalize the query
    query = normalize_variables(query)
    gt_result = graphdb_send_request(query)
    gt_results.append(gt_result)
    
    question = original_dataset[key]['Question']
    
    uris = original_dataset[key]['Gen_URI']
    gen_query1 = generate_URI_injected_query(question, uris)
    gen_query2 = generate_query(question, query)
    # Normalize the query
    gen_result1 = graphdb_send_request(gen_query1)
    gen_result2 = graphdb_send_request(gen_query2)
    gen_results.append(gen_result1)
    
    if gen_result1 and gt_result:
        comparisson = compare_csv_columns(gt_result, gen_result1)
        if comparisson == True:  
            detailed_comp1 += 1
        print(f"URI: {detailed_comp1}")
        
    if gen_result2 and gt_result:
        comparisson = compare_csv_columns(gt_result, gen_result2)
        if comparisson == True:  
            detailed_comp2 += 1
        print(f"PLAIN: {detailed_comp2}")
        
print (detailed_comp/100)